In [1]:
import pandas as pd
import numpy as np
import datetime
import sys
import MySQLdb
import matplotlib.pyplot as plt 
import matplotlib.dates as dates
import seaborn as sns
from IPython.display import display 
from ipywidgets import interact, fixed, widgets, HBox
from os.path import exists 

%matplotlib inline



/home/ihs01/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
SERVER_IP = "10.10.0.81"
TICK_INIT = 10*60*60
RESAMP_INIT = 120
MAX_DF_LEN = 1
DELTA_SAMP_MIN = 200
RESAMP_MAX = 120000*30


In [3]:
def printnow(val):
    print(val)
    sys.stdout.flush()

def connectDbase(dbase):
    conn = MySQLdb.connect(host=SERVER_IP, user="ihs01", passwd="!ndustr!al13", db=dbase)
    cursor = conn.cursor()
    return cursor

def changeDbase(dbase, cursor):
    cursor.execute('use %s;'%dbase) 

def readQuery(query, cursor):
    cursor.execute(query)
    data = cursor.fetchall()

def firstTime(tableName, cursor):
    query = 'select date from %s order by date limit 1'%tableName
    cursor.execute(query)
    data = cursor.fetchall()
    data = data[0][0]
    sdata = data.strftime("%Y-%m-%d %H:%M:%S")
    return sdata

def lastTime(cursor, tableName):
    query = 'select date from %s order by date desc limit 1'%tableName
    cursor.execute(query)
    data = cursor.fetchall()
    data = data[0][0]
    sdata = data.strftime("%Y-%m-%d %H:%M:%S")
    return sdata

def getColName(cursor, tableName):
    cursor.execute('desc %s'%tableName)
    colName = [column[0] for column in cursor.fetchall()]
    colDict ={}
    for i,j in enumerate(colName):
        colDict.update({i:j})
    return colDict

def lastTimeDF(df):
    return df.index[-1].strftime("%Y-%m-%d %H:%M:%S") 

def firstTimeDF(df):
    return df.index[0].strftime("%Y-%m-%d %H:%M:%S") 


In [4]:
def initDataFrame(data, colName):
    if len(data) == 0:
        df = pd.DataFrame(columns=colName)
    else:
        # create data frame with headers and date for index
        df = pd.DataFrame([[ij for ij in i] for i in data])
        df.rename(columns=colName, inplace=True);
        df.set_index(pd.to_datetime(df['date'].values), inplace=True)
    return df    
    
def readTable(cursor, tableName, query = 'SELECT * FROM %s'):
    # get col headers into a dictionary from mysql database
    colName = getColName(cursor, tableName)
    # execute query   
    cursor.execute(query%tableName)
    data = cursor.fetchall()
    df = initDataFrame(data, colName)
    return df

def readPartTable(cursor, tableName, start='2015-12-12 11:10:23', end='2015-12-12 13:10:23'):
    someCols =    ['date', 'atc1_mean', 'atc2_mean', 'atc3_mean', 'atc4_mean', 'atc5_mean', 'atc6_mean',
                  'roomtc1_mean', 'roomtc2_mean', 'fridgec_rms', 'fridgev_rms', 
                  'rtc1a_mean', 'rtc2a_mean', 'rtc3a_mean', 'rtc4a_mean',
                  'rtc5a_mean', 'rtc6a_mean', 'rtc7a_mean', 'rtc8a_mean',
                  'pressa_mean', 'htrva_rms', 'htria_rms', 'pidseta', 'pidtca', 'fpidset', 'fpidtc',  
                  'rtc1b_mean', 'rtc2b_mean', 'rtc3b_mean', 'rtc4b_mean',
                  'rtc5b_mean', 'rtc6b_mean', 'rtc7b_mean', 'rtc8b_mean',
                  'pressb_mean',
                  'htrib_rms', 'htrvb_rms', 'pidsetb', 'pidtcb']
    qString = ''
    for i in someCols:
        qString = qString + ('%s,'%(i)) 

    qString = qString[0:-1]    
    colName = dict(zip(np.arange(len(someCols)), someCols))
    #print 'SELECT %s FROM %s WHERE date BETWEEN \'%s\' and \'%s\''%(qString,tableName,start,end)
    cursor.execute('SELECT %s FROM %s WHERE date BETWEEN \'%s\' and \'%s\''%(qString,tableName,start,end))
    data = cursor.fetchall()
    df = initDataFrame(data, colName)
    return df

def rmsReadTable(cursor, tableName, blockLenInSec, start='2015-12-12 11:10:23', end='2015-12-12 11:20:23'):    
    colName = getColName(cursor, tableName)
    query = '''
    SELECT date, SQRT(SUM(POW(htrv,2))/COUNT(htrv)), 
           SQRT(SUM(POW(htri,2))/COUNT(htri)), SQRT(SUM(POW(fridgev,2))/COUNT(fridgev)), 
           SQRT(SUM(POW(fridgei,2))/COUNT(fridgei))
    FROM %s 
    WHERE date between \'%s\' AND \'%s\'
    GROUP BY unix_timestamp(date) div %d
    '''%(tableName, start, end, blockLenInSec)
    out = readQuery(query)
    df = initDataFrame(out, colName)
    return df

def getNextRMSTimeBlock(cursor, df, tablename, blockLenInSec):
    start = lastTimeDF(df)
    end = lastTime(cursor, tablename)
    new  = rmsReadTable(cursor, tablename, 120, start, end)
    if len(new) > 0:
        # get rid of first row
        new.drop(new.index[0], inplace=True)
    return new

def getNextTimeBlock(cursor, df, tablename):
    start = lastTimeDF(df)
    end = lastTime(cursor, tablename)
    new  = readPartTable(cursor, tablename, start, end)
    if len(new) > 0:
        # get rid of first row
        new.drop(new.index[0], inplace=True)
        df = df.append(new)
    return df

def getOneRMSBlock(cursor, tablenName, start, end):
    top = '''
           select  *
           from    %s
           '''
    bot = '''where   date between \'%s\' and \'%s\' '''%(start.strftime("%Y-%m-%d %H:%M:%S"),
                                                        end.strftime("%Y-%m-%d %H:%M:%S"))
    query = top + bot
    df = readTable(tableName, query)
    df.drop(['seconds', 'date'], axis=1, inplace=True)
    N = len(df)
    df = df **2    
    out = np.sqrt(df.sum() / float(N))
    out = out.to_frame().transpose()
    df.ix[0,:] = out.values    
    return df

In [5]:
def resampleRMS(array_like):
    return np.sqrt(np.sum(np.power(array_like,2))/float(len(array_like)))

def resampleMeanPlot(dfIn, col, title, dt=RESAMP_INIT, tickInt=TICK_INIT):
    # BoxCar Average on resample
    dt = str(dt) + 's'
    out = dfIn[col].resample(dt,how='mean')
    rng = pd.to_datetime(out.index)
    fig = plt.figure(figsize=[20,12])
    ax = fig.add_subplot(111)
    ax.plot_date(rng.to_pydatetime(), out, 'v-', lw=2.5, marker='o')
    
    # if your x tick labels are getting overlapped change interval on this line
    ax.xaxis.set_minor_locator(dates.SecondLocator(bysecond=None, interval=tickInt, tz=None))
    ax.xaxis.set_minor_formatter(dates.DateFormatter('%H:%M'))
    ax.xaxis.grid(True, which="minor")
    ax.yaxis.grid(True)
    ax.xaxis.set_major_locator(dates.WeekdayLocator(byweekday=(dates.MO,dates.TU,dates.WE,
                                                               dates.TH,dates.FR,dates.SA,dates.SU), interval=1))
    ax.xaxis.set_major_formatter(dates.DateFormatter('\n\n%d %a'))
    
    plt.title(title, fontsize=28, ha="center")  
    plt.tick_params(axis='y', labelsize=18)
    plt.tick_params(which='minor',axis='x', labelsize=18)
    plt.tick_params(which='major', axis='x', labelsize=24)
    plt.legend(col, fontsize=18)
    plt.tight_layout()
    plt.show()
    return fig

def resampleRMSPlot(dfIn, col, title, dt=RESAMP_INIT, tickInt=TICK_INIT):
    # Make sure your axis ticks are large enough to be easily read.    
    dt = str(dt) + 's'
    out = dfIn[col].resample(dt,how=resampleRMS)
    rng = pd.to_datetime(out.index)

    fig, ax = plt.subplots(figsize=[20,12])
    ax.plot_date(rng.to_pydatetime(), out, 'v-', lw=2.5, marker='o')
    # if your x tick labels are getting overlapped change interval on this line
    ax.xaxis.set_minor_locator(dates.SecondLocator(bysecond=None, interval=tickInt, tz=None))
    ax.xaxis.set_minor_formatter(dates.DateFormatter('%H:%M'))
    ax.xaxis.grid(True, which="minor")
    ax.yaxis.grid(True)
    ax.xaxis.set_major_locator(dates.WeekdayLocator(byweekday=(dates.MO,dates.TU,dates.WE,dates.TH,dates.FR,dates.SA,dates.SU), interval=1))
    
    ax.xaxis.set_major_formatter(dates.DateFormatter('\n\n%d %a'))
    
    plt.title(title, fontsize=28, ha="center")  
    plt.tick_params(axis='y', labelsize=18)
    plt.tick_params(which='minor',axis='x', labelsize=18)
    plt.tick_params(which='major', axis='x', labelsize=24)
    plt.legend(col, fontsize=18)
    plt.tight_layout()
    plt.show()
    return fig


In [6]:
def calcPower(df1):
    df1['pha']     = df1['htrva_rms'] * df1['htria_rms']
    df1['phb']     = df1['htrvb_rms'] * df1['htrib_rms']
    df1['pf']      = df1['fridgev_rms'] * df1['fridgec_rms']            
    df1['eha']     = df1['pha'].cumsum()
    df1['ehb']     = df1['phb'].cumsum()
    df1['ef']      = df1['pf'].cumsum()
    df1['ratioa']  = df1['eha'] / df1['ef']
    df1['ratiob']  = df1['ehb'] / df1['ef']
    return df1

def update(cursor,df1, tableName, fileName):
    global NEW_TABLE
    
    df1 = getNextTimeBlock(cursor, df1, tableName)
    if NEW_TABLE >=7:
        df1 = calcPower(df1)
    
    print "Updated Data"
    print 'Last Database Entry:  %s'%lastTime(cursor, tableName)
    print 'Last Dataframe Entry: %s'%lastTimeDF(df1)
    pickleDataFrame(df1, fileName)    
    
    return df1

def pickleDataFrame(df, name):
    df.to_pickle(name)
    return

def readDataFrame(name):
    df = pd.read_pickle(name)
    return df

def plotAll(df1, resampRate=RESAMP_INIT, tick=TICK_INIT):
    global NEW_TABLE
    
    if NEW_TABLE >=9:
        f = resampleMeanPlot(df1, ['atc1_mean','atc2_mean','atc3_mean','atc4_mean','atc5_mean','atc6_mean'], 
                         'Thermal Chamber Temp $^\circ$C ', resampRate, tick);
        f = resampleMeanPlot(df1, ['rtc1a_mean','rtc2a_mean','rtc3a_mean','rtc4a_mean',
                               'rtc5a_mean','rtc6a_mean','rtc7a_mean','rtc8a_mean'], 
                         'RCT A Reacter Skin Temp $^\circ$C', resampRate, tick);
        f = resampleMeanPlot(df1, ['rtc1b_mean','rtc2b_mean','rtc3b_mean','rtc4b_mean',
                               'rtc5b_mean','rtc6b_mean','rtc7b_mean','rtc8b_mean'], 
                         'RCT B Reacter Skin Temp $^\circ$C', resampRate, tick);
        f = resampleMeanPlot(df1, ['pressa_mean'], 'RCT A Pressure psi', resampRate, tick);
        f = resampleMeanPlot(df1, ['pressb_mean'], 'RCT B Pressure psi', resampRate, tick);
        f = resampleMeanPlot(df1, ['pidseta','pidtca'], 
                         'RCT A Core Temp and PID Set Point $^\circ$C', resampRate, tick);
        f = resampleMeanPlot(df1, ['pidsetb','pidtcb'], 
                         'RCT B Core Temp and PID Set Point $^\circ$C', resampRate, tick);
        f = resampleMeanPlot(df1, ['fpidset', 'fpidtc'], 
                         'Thermal Chamber Temp and PID Set Point $^\circ$C', resampRate, tick);
        f = resampleMeanPlot(df1, ['roomtc1_mean','roomtc2_mean'], 
                         'RCT B Lab Ambient Temp $^\circ$C', resampRate, tick);
        f = resampleRMSPlot(df1, ['htrva_rms'], 'RCT A Heater Voltage (V)', resampRate, tick);
        f = resampleRMSPlot(df1, ['htrvb_rms'], 'RCT B Heater Voltage (V)', resampRate, tick);
        f = resampleRMSPlot(df1, ['fridgev_rms'], 'RCT A Thermal Chamber Voltage (V)', resampRate, tick);
        f = resampleRMSPlot(df1, ['htria_rms'], 'RCT A Heater Current (A)', resampRate, tick);
        f = resampleRMSPlot(df1, ['htrib_rms'], 'RCT B Heater Current (A)', resampRate, tick);
        f = resampleRMSPlot(df1, ['fridgec_rms'], 'RCT A Thermal Chamber Current (A)', resampRate, tick);

        f = resampleRMSPlot(df1, ['pha'], 'RCT A Heater Power (W)', resampRate, tick);
        f = resampleRMSPlot(df1, ['phb'], 'RCT B Heater Power (W)', resampRate, tick);
        f = resampleRMSPlot(df1, ['pf'], 'Thermal Chamber Power (W)', resampRate, tick);

        f = resampleRMSPlot(df1, ['eha'], 'RCT A Cumulative Energy (~J)', resampRate, tick);
        f = resampleRMSPlot(df1, ['ehb'], 'RCT B Cumulative Energy (~J)', resampRate, tick);
        f = resampleRMSPlot(df1, ['ef'], 'Thermal Chamber Cumulative Energy (~J)', resampRate, tick);
        f = resampleRMSPlot(df1, ['ratioa', 'ratiob'], 'RCT A/B : Thermal Chamber Energy Ratio', resampRate, tick);  
        
    else:
        print 'Table %d Not Supported'.format(NEW_TABLE)

In [7]:
def slidePlot(dfIn, signal=('atc1_mean',), resample_rate_in_sec=RESAMP_INIT, 
              start=0, end=MAX_DF_LEN, resampMethod='rms'):
    dfIn = dfIn.ix[start:end,:]
    col = list(signal)
    title = signal
    dt = str(resample_rate_in_sec) + 'ms'
    if resampMethod == 'rms' : resampMethod = resampleRMS

    # get x,y for plot
    out = dfIn[col].resample(dt,how=resampMethod)
    rng = pd.to_datetime(out.index)

    fig, ax = plt.subplots(1,1, figsize=[20,12])
    ax.plot_date(rng.to_pydatetime(), out, 'v-', lw=2.5, marker='o')
    fig.autofmt_xdate()

    ax.xaxis.set_major_formatter(dates.DateFormatter('%a %m/%d-%H:%M:%S'))
    ax.xaxis.grid(True)
    ax.yaxis.grid(True)

    plt.title(title, fontsize=28, ha="center")  
    plt.tick_params(axis='y', labelsize=18)
    plt.tick_params(which='minor',axis='x', labelsize=18)
    plt.tick_params(which='major', axis='x', labelsize=18)
    plt.legend(col, fontsize=18, loc='best')
    plt.tight_layout()
    fig.canvas.draw()
    return 

def iSlidePlot(df, MAX_RESAMP_TIME=RESAMP_MAX, DELTA_SAMP=DELTA_SAMP_MIN, DELTA_TIME=1):
    
    if df.empty:
        print 'Null DataFrame'
    else:

        %matplotlib inline
    
        attributes = df.columns.tolist()
        MAX_NUM = len(df)-2

        sigWid = widgets.SelectMultiple(
        description = "Signal",
        options=df.columns.tolist()[1:],
        value=('atc1_mean','atc2_mean','atc3_mean',
               'atc4_mean','atc5_mean','atc6_mean')
        )

        resampMethodWid = widgets.RadioButtons(
        description='Resample Method',
        options=['mean', 'rms'],
        value='mean',
        )

        resampRateWid = widgets.IntSlider(
        value=120000,
        min=DELTA_SAMP,
        max=MAX_RESAMP_TIME,
        step=DELTA_SAMP,
        description='Resample Rate (ms)', 
        continuous_update=False,
        )

        startWid = widgets.IntSlider(
        value=0,
        min=0,
        max=len(df),
        step=DELTA_TIME,
        description='Start Sample', 
        continuous_update=False,
        )

        endWid = widgets.IntSlider(
        value=len(df),
        min=0,
        max=len(df),
        step=DELTA_TIME,
        description='End Sample', 
        continuous_update=False,
        )
    
        interact(slidePlot, dfIn=fixed(df), 
             signal=sigWid, 
             resample_rate_in_sec=resampRateWid, 
             start=startWid, 
             end=endWid,
             resampMethod = resampMethodWid)
    return


In [8]:
tableWid = widgets.Dropdown(
    description='Run/Table',
    options={'Table 005' : 5,
             'Table 007' : 7,
             'Table 006' : 6,
             'Table 008' : 8,
             'Table 009' : 9,
             'Table 010' : 10},
    value=10,
)


reactorWid = widgets.Dropdown(
    description='Reactor/dBase',
    options={'Reactor 005' : 5},
    value=5,
)


updateWid = widgets.Checkbox(
description='Update Data',
value=False
)
    
table = tableWid
dBase = reactorWid
new = updateWid
ChooseBox = HBox([dBase, table, new])
df = pd.DataFrame()
OLD_TABLE = 0
NEW_TABLE = 0

def initConsole(tbl, rct, getData, df):
    tbl = tbl.value
    rct = rct.value
    getData = getData.value
    global OLD_TABLE
    global NEW_TABLE
    
    NEW_TABLE = tbl
    TABLE = '{0:03d}'.format(tbl)
    RCT = '{0:03d}'.format(rct)
    pickleFile = '../DATA/rct' + RCT + 'run' + TABLE + '.pkl'
    dbName = 'rct' + RCT[1:] + 'db'
    tbName = TABLE + '_data'

    if NEW_TABLE!=OLD_TABLE:
        # if file exist return it or else do a dbase read
        OLD_TABLE = tbl
        if exists(pickleFile):
            df = readDataFrame(pickleFile)
        else:
            default_start = '1999-12-28 18:00:00'
            time_now = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            cursor = connectDbase(dbName)
            df = readPartTable(cursor,tbName, default_start, time_now)
    
    elif getData:
        cursor = connectDbase(dbName)
        df = update(cursor, df, tbName, pickleFile)    
            
    return df